# Semantické jadro s integráciou OpenBnB MCP servera

Tento notebook ukazuje, ako používať Semantické jadro s reálnym OpenBnB MCP serverom na vyhľadávanie skutočných ubytovaní Airbnb pomocou MCPStdioPlugin. Na prístup k LLM využíva Azure AI Foundry. Na nastavenie vašich environmentálnych premenných môžete postupovať podľa [Lekcie nastavenia](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Vytvorenie pripojenia MCP Plugin

Pripojíme sa k [OpenBnB MCP serveru](https://github.com/openbnb-org/mcp-server-airbnb) pomocou MCPStdioPlugin. Tento server poskytuje funkciu vyhľadávania Airbnb prostredníctvom balíka @openbnb/mcp-server-airbnb.


## Vytvorenie klienta

V tomto príklade použijeme Azure AI Foundry na prístup k LLM. Uistite sa, že vaše premenné prostredia sú správne nastavené.


## Konfigurácia prostredia

Nakonfigurujte nastavenia Azure OpenAI. Uistite sa, že máte nastavené nasledujúce premenné prostredia:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Porozumenie integrácii OpenBnB MCP

Tento notebook sa pripája k **skutočnému serveru OpenBnB MCP**, ktorý poskytuje funkciu vyhľadávania na Airbnb.

### Ako to funguje:

1. **MCPStdioPlugin**: Používa komunikáciu cez štandardný vstup/výstup so serverom MCP
2. **Skutočný NPM balík**: Stiahne a spustí `@openbnb/mcp-server-airbnb` pomocou npx
3. **Živé dáta**: Poskytuje skutočné údaje o nehnuteľnostiach z Airbnb API
4. **Objavovanie funkcií**: Agent automaticky objavuje dostupné funkcie zo servera MCP

### Dostupné funkcie:

Server OpenBnB MCP zvyčajne poskytuje:
- **search_listings** - Vyhľadávanie nehnuteľností na Airbnb podľa lokality a kritérií
- **get_listing_details** - Získanie podrobných informácií o konkrétnych nehnuteľnostiach
- **check_availability** - Kontrola dostupnosti na konkrétne dátumy
- **get_reviews** - Získanie recenzií na nehnuteľnosti
- **get_host_info** - Získanie informácií o hostiteľoch nehnuteľností

### Predpoklady:

- **Node.js** nainštalovaný vo vašom systéme
- **Internetové pripojenie** na stiahnutie balíka servera MCP
- **NPX** dostupné (je súčasťou Node.js)

### Testovanie pripojenia:

Server MCP môžete otestovať manuálne spustením:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Tým sa stiahne a spustí server OpenBnB MCP, ku ktorému sa potom Semantic Kernel pripojí pre skutočné údaje z Airbnb.


## Spustenie agenta s OpenBnB MCP serverom

Teraz spustíme AI agenta, ktorý sa pripája k OpenBnB MCP serveru, aby vyhľadal skutočné ubytovanie na Airbnb v Štokholme pre 2 dospelých a 1 dieťa. Neváhajte upraviť zoznam `user_inputs`, aby ste zmenili kritériá vyhľadávania.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Zhrnutie
Gratulujeme! Úspešne ste vytvorili AI agenta, ktorý integruje vyhľadávanie ubytovania v reálnom svete pomocou Model Context Protocol (MCP):

## Použité technológie:
- Semantic Kernel - Na vytváranie inteligentných agentov s Azure OpenAI
- Azure AI Foundry - Na schopnosti LLM a dokončovanie konverzácií
- MCP (Model Context Protocol) - Na štandardizovanú integráciu nástrojov
- OpenBnB MCP Server - Na skutočné vyhľadávanie ubytovania cez Airbnb
- Node.js/NPX - Na spustenie externého MCP servera

## Čo ste sa naučili:
- Integrácia MCP: Prepojenie agentov Semantic Kernel s externými MCP servermi
- Prístup k dátam v reálnom čase: Vyhľadávanie skutočných nehnuteľností cez živé API
- Komunikácia cez protokol: Používanie stdio komunikácie medzi agentom a MCP serverom
- Objavovanie funkcií: Automatické objavovanie dostupných funkcií z MCP serverov
- Streamovanie odpovedí: Zachytávanie a zaznamenávanie volaní funkcií v reálnom čase
- HTML vykresľovanie: Formátovanie odpovedí agenta pomocou štýlovaných tabuliek a interaktívnych zobrazení

## Ďalšie kroky:
- Integrácia ďalších MCP serverov (počasie, lety, reštaurácie)
- Vytvorenie systému viacerých agentov kombinujúceho MCP a A2A protokoly
- Vytvorenie vlastných MCP serverov pre vaše vlastné zdroje dát
- Implementácia pamäte konverzácií naprieč reláciami
- Nasadenie agenta do Azure Functions s orchestráciou MCP serverov
- Pridanie autentifikácie používateľov a schopností rezervácie



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
